In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [3]:
df=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
num_columns = len(df.columns)
print(f"Number of columns using len(df.columns): {num_columns}")

Number of columns using len(df.columns): 19


In [6]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [7]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [8]:
# Calculate the standard deviation of the trips duration in January
std_dev_january_duration = df['duration'].std()

print(f"Standard deviation of trip duration in January: {std_dev_january_duration:.2f} minutes")

Standard deviation of trip duration in January: 42.59 minutes


In [9]:
print("Original DataFrame head:")
print(df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'duration']].head(10))
print(f"\nOriginal number of records: {len(df)}")

# Store the initial number of records
initial_records_count = len(df)

# Filter out the outliers: keep duration between 1 and 60 minutes (inclusive)
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

# Get the number of records after filtering
records_after_filtering_count = len(df_filtered)

# Calculate the fraction of records left
fraction_left = records_after_filtering_count / initial_records_count

print(f"\nNumber of records after dropping outliers: {records_after_filtering_count}")
print(f"Fraction of records left: {fraction_left:.4f}")

Original DataFrame head:
  tpep_pickup_datetime tpep_dropoff_datetime   duration
0  2023-01-01 00:32:10   2023-01-01 00:40:36   8.433333
1  2023-01-01 00:55:08   2023-01-01 01:01:27   6.316667
2  2023-01-01 00:25:04   2023-01-01 00:37:49  12.750000
3  2023-01-01 00:03:48   2023-01-01 00:13:25   9.616667
4  2023-01-01 00:10:29   2023-01-01 00:21:19  10.833333
5  2023-01-01 00:50:34   2023-01-01 01:02:52  12.300000
6  2023-01-01 00:09:22   2023-01-01 00:19:49  10.450000
7  2023-01-01 00:27:12   2023-01-01 00:49:56  22.733333
8  2023-01-01 00:21:44   2023-01-01 00:36:40  14.933333
9  2023-01-01 00:39:42   2023-01-01 00:50:36  10.900000

Original number of records: 3066766

Number of records after dropping outliers: 3009173
Fraction of records left: 0.9812


In [10]:
from sklearn.feature_extraction import DictVectorizer
print("Original DataFrame head:")
print(df.head())

# 1. Select only the relevant features
# We'll work with a subset of the DataFrame containing only these two columns.
df_locations = df[['PULocationID', 'DOLocationID']]

# 2. Turn the DataFrame into a list of dictionaries,
#    remembering to re-cast the IDs to strings.
#    This is crucial for DictVectorizer to treat them as categorical features.
dict_list = df_locations.apply(
    lambda row: {
        'PULocationID': str(row['PULocationID']),
        'DOLocationID': str(row['DOLocationID'])
    },
    axis=1
).tolist()

# print("\nSample of the list of dictionaries (first 5):")
# print(dict_list[:5])

# 3. Fit a dictionary vectorizer
vec = DictVectorizer()

# 4. Get a feature matrix from it
feature_matrix = vec.fit_transform(dict_list)

# 5. What's the dimensionality of this matrix (number of columns)?
dimensionality = feature_matrix.shape[1]

print(f"\nDimensionality of the feature matrix (number of columns): {dimensionality}")

# You can also inspect the feature names to understand what columns were created
# print("\nFeature names generated by DictVectorizer:")
# print(vec.get_feature_names_out())

Original DataFrame head:
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  fare_amount  extra  mta_tax  tip